## Importar urls 

In [2]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW 
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio import SeqFeature
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation

In [2]:
import time
def url_get(i):
    url_list= [ ]
    for id in range(7200,(7200+i)):
        url = "https://www.ncbi.nlm.nih.gov/nuccore/NC_06{}".format( id )
        url_list.append(url)
    return url_list

url_get(1)

['https://www.ncbi.nlm.nih.gov/nuccore/NC_067200']

## Importar requests

In [3]:
import requests

content = []
for url in url_get(1):
    r = requests.get(url)
    content.append(r.content)
print(content)

[b'<?xml version="1.0" encoding="utf-8"?>\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en">\n    <head xmlns:xi="http://www.w3.org/2001/XInclude"><meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n    <!-- meta -->\n    <meta name="robots" content="index,nofollow,noarchive" />\n<meta name="ncbi_app" content="entrez" /><meta name="ncbi_db" content="nuccore" /><meta name="ncbi_report" content="genbank" /><meta name="ncbi_format" content="html" /><meta name="ncbi_pagesize" content="20" /><meta name="ncbi_sortorder" content="default" /><meta name="ncbi_pageno" content="1" /><meta name="ncbi_resultcount" content="1" /><meta name="ncbi_op" content="retrieve" /><meta name="ncbi_pdid" content="genbank" /><meta name="ncbi_sessionid" content="CE89A3AF3BAB2DA1_0636SID" /><meta name="ncbi_uidlist" content="2318775852" /><meta name="ncbi_f

## Importar ficheiro genbanck

In [16]:
import requests
from bs4 import BeautifulSoup
import re

# Parsing the HTML
with open('genbank_new', 'w') as f:
    for c in content:
        soup = BeautifulSoup(c, 'html.parser')

        # Procurar um tag meta com um determinado atributo
        lines = soup.find_all('meta', {'name':"ncbi_uidlist"} )

        id = ""
        url = ""
        for line in lines:
            #print(line)
            #if 'name' in line.attrs:
            #    print(line.attrs['name'])
            if 'content' in line.attrs:
                # print(line.attrs['content'])		
                id = line.attrs['content']

        if id:
            url = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000".format(id)

        r2 = requests.get( url )
        #print( r2.content, "\n" )
        f.write(print(r2.content, "\n"))
   

b'LOCUS       NC_067200           22148653 bp    DNA     linear   CON 18-OCT-2022\nDEFINITION  Schistosoma haematobium chromosome 5, whole genome shotgun\n            sequence.\nACCESSION   NC_067200\nVERSION     NC_067200.1\nDBLINK      BioProject: PRJNA273970\n            BioSample: SAMN11431408\n            Assembly: GCF_000699445.3\nKEYWORDS    WGS; RefSeq.\nSOURCE      Schistosoma haematobium\n  ORGANISM  Schistosoma haematobium\n            Eukaryota; Metazoa; Spiralia; Lophotrochozoa; Platyhelminthes;\n            Trematoda; Digenea; Strigeidida; Schistosomatoidea;\n            Schistosomatidae; Schistosoma.\nREFERENCE   1  (bases 1 to 22148653)\n  AUTHORS   Stroehlein,A.J., Korhonen,P.K., Chong,T.M., Lim,Y.L., Chan,K.G.,\n            Webster,B., Rollinson,D., Brindley,P.J., Gasser,R.B. and Young,N.D.\n  TITLE     High-quality Schistosoma haematobium genome achieved by\n            single-molecule and long-range sequencing\n  JOURNAL   Gigascience 8 (9) (2019)\n   PUBMED   31494

TypeError: write() argument must be str, not None

### Outra maneira (que eu prefiro)

In [3]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SwissProt import KeyWList
from Bio import SwissProt
from Bio import ExPASy
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio import Phylo

In [5]:
database = 'nucleotide'
word = 'GLI '
res= input()
email= 'rodrigoce9@gmail.com'
Entrez.email= email
handle_search=Entrez.esearch(db = database, term=word, retmax= res)
record=Entrez.read(handle_search)
handle_search.close()
idlist= record['IdList']
print (record)


2
{'Count': '10706', 'RetMax': '2', 'RetStart': '0', 'IdList': ['2421755792', '2421750421'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'GLI[All Fields]', 'Field': 'All Fields', 'Count': '10706', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'GLI[All Fields]'}


In [3]:
Ids= []
Pubmed=[]
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
#featcds = [ ]
position=0
pos=[]
for info in records:
    Ids.append(info.id)
    for i in range(len(info.features)):
        if info.features[i].type == "CDS":
            #featcds.append(i)
            cds= i
            print (len(cds))
            position=i
            pos.append(info.features[position].location)
#print (featcds)
#print(pos)
    
    #print('length of seq:', len(info.seq)) #to check the length of the sequences

TypeError: object of type 'int' has no len()

In [62]:
import os
Entrez.email = "rodrigoce9@gmail.com"
for x in Ids:
    filename = x
    if not os.path.isfile(filename):
        net_handle = Entrez.efetch(db="nucleotide", id=x, rettype="gb", retmode="text")
        out_handle = open(filename, "w")
        out_handle.write(net_handle.read())
        out_handle.close()
        net_handle.close()

In [63]:
for i in Ids:
    database = "PubMed"
    res= 1
    word = i
    email= "rodrigoce9@gmail.com"

    Entrez.email= email
    handle=Entrez.esearch(db = database, term=word, retmax= res)
    record=Entrez.read(handle)
    handle.close()
    idlist= record['IdList']

    handle = Entrez.efetch(db=database, id=idlist, rettype="medline", retmode="text")
    records = Medline.parse(handle)

    for record in records:
        #print(record) 
        #print("PMID:",record.get("PMID","-"))
        print("title:", record.get("TI", "-"))
        #print('abstract:', record.get('AB', '-'))
        print("authors:", record.get("AU", "-"))
        print("source:", record.get("SO", "-"))
        print("")

title: The serum oestradiol/progesterone ratio on the day of OPU + 7, but not the day of OPU + 5, affects the rates of live birth in fresh blastocyst embryo transfer cycles.
authors: ['Zhao W', 'Diao H', 'Chen X', 'Xu S', 'Jiang S', 'Cao H', 'Zhang C', 'Zhang Y']
source: J Ovarian Res. 2023 Jan 7;16(1):4. doi: 10.1186/s13048-023-01096-3.

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

title: -
authors: -
source: -

